# Import Libraries

In [1]:
# feature extractoring and preprocessing data
import librosa as librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
import soundfile as sf
from glob import glob
import IPython.display as ipd

import io
from urllib.request import urlopen


# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler


#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [2]:
# test with libraries
# Download https://www.kaggle.com/kinguistics/heartbeat-sounds/data

# data_dir = './music_genre_dataset/heartbeat-sounds/set_a'
# audio_files = glob(data_dir+'/*.wav')
# len(audio_files)

In [3]:
# for file in range(0, len(audio_files), 1):
#     audio, sfreq = librosa.load(audio_files[file])
#     time = np.arange(0, len(audio))/sfreq
    
#     fig, ax = plt.subplots()
#     ax.plot(time,audio)
#     ax.set(xlabel= 'Time em segundos', ylabel = "Amplitude")
#     plt.show()
    

In [4]:
#     X = librosa.stft(audio)
#     Xdb = librosa.amplitude_to_db(abs(X))
#     plt.figure(figsize=(14, 5))
#     librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
#     plt.colorbar()

In [5]:
#  librosa.output.write_wav('example.wav', x, sr)

# Extracting and feature music

## Dataset
We use GTZAN genre collection dataset for classification.

The dataset consists of 10 genres,each genre contains 100 songs. Total dataset: 1000 songs

 - Blues
 - Classical
 - Country
 - Disco
 - Hiphop
 - Jazz
 - Metal
 - Pop
 - Reggae
 - Rock


In [6]:
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    data_dir = f'music_genre_dataset/genres/{g}'
    audio_files = glob(data_dir+'/*.wav')
    print(f'Total musics in genre {g}: ', len(audio_files))

Total musics in genre blues:  100
Total musics in genre classical:  100
Total musics in genre country:  100
Total musics in genre disco:  100
Total musics in genre hiphop:  100
Total musics in genre jazz:  100
Total musics in genre metal:  100
Total musics in genre pop:  100
Total musics in genre reggae:  100
Total musics in genre rock:  100


In [7]:
# genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
# for g in genres:
#     data_dir = f'music_genre_dataset/genres/{g}'
#     audio_files = glob(data_dir+'/*.wav')
    
#     print('--------------------------------------------------------------------------------')
#     print(f'List musics in genre: {g}')
    
#     for file in range(0, len(audio_files), 1):
#         audio, sfreq = librosa.load(audio_files[file])
#         time = np.arange(0, len(audio))/sfreq
#         fig, ax = plt.subplots()
#         ax.plot(time,audio)
#         ax.set(xlabel= 'Time em segundos', ylabel = "Amplitude")
#         plt.show()

# Extracting the Spectrogram for every Audio

In [8]:
# cmap = plt.get_cmap('inferno')

# plt.figure(figsize=(10,10))
# genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
# for g in genres:
#     data_dir = f'music_genre_dataset/genres/{g}'
#     audio_files = glob(data_dir+'/*.wav')
    
#     print('--------------------------------------------------------------------------------')
#     print(f'List musics in genre: {g}')
    
    
#     for filename in os.listdir(f'./music_genre_dataset/genres/{g}'):
#         songname = f'./music_genre_dataset/genres/{g}/{filename}'
        
#         y, sr = librosa.load(songname, mono=True, duration=5)
#         plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
#         plt.axis('off');
#         plt.savefig(f'./music_genre_dataset/img_data/{g}/{filename[:-3].replace(".", "")}.png')
#         plt.clf()                    

All the audio files get converted into their respective spectrograms .WE can noe easily extract features from them.

# Extracting features from Spectrogram

We will extract

 - Mel-frequency cepstral coefficients (MFCC)(20 in number)
 - Spectral Centroid,
 - Zero Crossing Rate
 - Chroma Frequencies
 - Spectral Roll-off.

In [9]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

# Writing data to csv file¶
We write the data to a csv file

In [10]:
file = open('./music_genre_dataset/data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
#     data_dir = f'music_genre_dataset/genres/{g}'
    
    for filename in os.listdir(f'./music_genre_dataset/genres/{g}'):
        songname = f'./music_genre_dataset/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{file} {np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('./music_genre_dataset/data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())
        

In [11]:
# file = open('./music_genre_dataset/data.csv', 'w', newline='')
# with file:
#     writer = csv.writer(file)
#     writer.writerow(header)
# genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
# for g in genres:
#     print(os.listdir(f'./music_genre_dataset/genres/{g}')
        
    
#     for filename in os.listdir(f'./music_genre_dataset/genres/{g}'):
#         songname = f'./music_genre_dataset/genres/{g}/{filename}'
#         y, sr = librosa.load(songname, mono=True, duration=30)
#         chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
#         spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
#         spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
#         rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
#         zcr = librosa.feature.zero_crossing_rate(y)
#         mfcc = librosa.feature.mfcc(y=y, sr=sr)
#         to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
#         for e in mfcc:
#             to_append += f' {np.mean(e)}'
#         to_append += f' {g}'
#         file = open('./music_genre_dataset/data.csv', 'a', newline='')
#         with file:
#             writer = csv.writer(file)
#             writer.writerow(to_append.split())

In [12]:
# Data Analising with padas

In [13]:
data = pd.read_csv('./music_genre_dataset/data.csv')
data.head()

filename  \
<_io.TextIOWrapper name='./music_genre_dataset/data.csv'  mode='w'   
                   name='./music_genre_dataset/data.csv'  mode='a'   
                   name='./music_genre_dataset/data.csv'  mode='a'   
                   name='./music_genre_dataset/data.csv'  mode='a'   
                   name='./music_genre_dataset/data.csv'  mode='a'   

                                                                chroma_stft  \
<_io.TextIOWrapper name='./music_genre_dataset/data.csv'  encoding='UTF-8'>   
                   name='./music_genre_dataset/data.csv'  encoding='UTF-8'>   
                   name='./music_genre_dataset/data.csv'  encoding='UTF-8'>   
                   name='./music_genre_dataset/data.csv'  encoding='UTF-8'>   
                   name='./music_genre_dataset/data.csv'  encoding='UTF-8'>   

                                                              rmse  \
<_io.TextIOWrapper name='./music_genre_dataset/data.csv'  0.329163   
                   name='./music_genre_dataset/data.csv'  0.377688   
                   name='./music_genre_dataset/data.csv'  0.302346   
                   name='./music_genre_dataset/data.csv'  0.400860   
                   name='./music_genre_dataset/data.csv'  0.387851   

                                                          spectral_centroid  \
<_io.TextIOWrapper name='./music_genre_dataset/data.csv'        1172.065695   
                   name='./music_genre_dataset/data.csv'        1613.315725   
                   name='./music_genre_dataset/data.csv'        1831.942368   
                   name='./music_genre_dataset/data.csv'        1945.848425   
                   name='./music_genre_dataset/data.csv'        1912.506881   

                                                          spectral_bandwidth  \
<_io.TextIOWrapper name='./music_genre_dataset/data.csv'         1705.817502   
                   name='./music_genre_dataset/data.csv'         1972.202261   
                   name='./music_genre_dataset/data.csv'         1729.483241   
                   name='./music_genre_dataset/data.csv'         2082.246626   
                   name='./music_genre_dataset/data.csv'         2361.474653   

                                                              rolloff  \
<_io.TextIOWrapper name='./music_genre_dataset/data.csv'  2344.952487   
                   name='./music_genre_dataset/data.csv'  3470.404015   
                   name='./music_genre_dataset/data.csv'  3480.937285   
                   name='./music_genre_dataset/data.csv'  4175.874749   
                   name='./music_genre_dataset/data.csv'  4776.862782   

                                                          zero_crossing_rate  \
<_io.TextIOWrapper name='./music_genre_dataset/data.csv'            0.045046   
                   name='./music_genre_dataset/data.csv'            0.058236   
                   name='./music_genre_dataset/data.csv'            0.094040   
                   name='./music_genre_dataset/data.csv'            0.085806   
                   name='./music_genre_dataset/data.csv'            0.058426   

                                                               mfcc1  \
<_io.TextIOWrapper name='./music_genre_dataset/data.csv' -305.705170   
                   name='./music_genre_dataset/data.csv' -177.490753   
                   name='./music_genre_dataset/data.csv' -177.869049   
                   name='./music_genre_dataset/data.csv'  -82.979019   
                   name='./music_genre_dataset/data.csv' -182.638275   

                                                               mfcc2  \
<_io.TextIOWrapper name='./music_genre_dataset/data.csv'  113.060928   
                   name='./music_genre_dataset/data.csv'  115.346100   
                   name='./music_genre_dataset/data.csv'  118.196907   
                   name='./music_genre_dataset/data.csv'  107.052124   
                   name='./music_genre_dataset/data.csv'   90.676796   


In [14]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

In [15]:
# Encode labels


genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [16]:
# Scaling feature columns


In [17]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

ValueError: could not convert string to float: "encoding='UTF-8'>"

In [ ]:
# Diving data into training and test


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
len(y_train)

In [ ]:
len(y_test)

In [ ]:
X_train[10]

# Classification with Keras


In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [ ]:

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

In [ ]:
test_loss, test_acc = model.evaluate(X_test,y_test)

In [ ]:
print('test_acc: ',test_acc)

# Validating out approuch


In [ ]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [ ]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

In [ ]:
results

# Predictions on Test Data

In [ ]:
predictions = model.predict(X_test)

In [ ]:

predictions[0].shape

In [ ]:

np.sum(predictions[0])

In [ ]:

np.argmax(predictions[0])